# Movie Analytics Project 

## Overview

Overview Here:

### Loading in and Investigating the Data

In [18]:
import pandas as pd
import sqlite3
import zipfile

movie_gross = pd.read_csv('bom.movie_gross.csv.gz', compression='gzip')
print(movie_gross.info())

with zipfile.ZipFile('im.db.zip', 'r') as zip_ref:
    zip_ref.extractall('.')

conn = sqlite3.connect('im.db')

tables = pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='table';", conn)
print(tables)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3387 entries, 0 to 3386
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   title           3387 non-null   object 
 1   studio          3382 non-null   object 
 2   domestic_gross  3359 non-null   float64
 3   foreign_gross   2037 non-null   object 
 4   year            3387 non-null   int64  
dtypes: float64(1), int64(1), object(3)
memory usage: 132.4+ KB
None
            name
0   movie_basics
1      directors
2      known_for
3     movie_akas
4  movie_ratings
5        persons
6     principals
7        writers


In [19]:
movie_gross.head()

,title,studio,domestic_gross,foreign_gross,year
0,Toy Story 3,BV,415000000.0,652000000,2010
1,Alice in Wonderland (2010),BV,334200000.0,691300000,2010
2,Harry Potter and the Deathly Hallows Part 1,WB,296000000.0,664300000,2010
3,Inception,WB,292600000.0,535700000,2010
4,Shrek Forever After,P/DW,238700000.0,513900000,2010


### Cleaning Movie Gross Dataset

In [20]:
# Converting 'foreign_gross' to numeric so it can be used in analysis
movie_gross['foreign_gross'] = pd.to_numeric(movie_gross['foreign_gross'].str.replace(',', ''), errors='coerce')

print(movie_gross.dtypes)

title              object
studio             object
domestic_gross    float64
foreign_gross     float64
year                int64
dtype: object


In [22]:
movie_gross.isnull().sum()

title                0
studio               5
domestic_gross      28
foreign_gross     1350
year                 0
dtype: int64

In [23]:
# Checking out the rows that have null values in studio
null_studio = movie_gross[movie_gross['studio'].isnull()]
print("Rows with null 'studio':")
print(null_studio)

Rows with null 'studio':
                              title studio  domestic_gross  foreign_gross  \
210   Outside the Law (Hors-la-loi)    NaN         96900.0      3300000.0   
555         Fireflies in the Garden    NaN         70600.0      3300000.0   
933           Keith Lemon: The Film    NaN             NaN      4000000.0   
1862                 Plot for Peace    NaN          7100.0            NaN   
2825               Secret Superstar    NaN             NaN    122000000.0   

      year  
210   2010  
555   2011  
933   2012  
1862  2014  
2825  2017  


In [25]:
# Manually updating the missing studios since there are only 5

studios = {
    210: 'Tessalit Productions',
    555: 'Senator Distribution',
    933: 'Lionsgate UK',
    1862: 'Indelible Media',
    2825: 'Aamir Khan Productions'
}

for index, studio in studios.items():
    movie_gross.at[index, 'studio'] = studio

updated_rows = movie_gross.loc[list(studios.keys())]
print(updated_rows)


                              title                  studio  domestic_gross  \
210   Outside the Law (Hors-la-loi)    Tessalit Productions         96900.0   
555         Fireflies in the Garden    Senator Distribution         70600.0   
933           Keith Lemon: The Film            Lionsgate UK             NaN   
1862                 Plot for Peace         Indelible Media          7100.0   
2825               Secret Superstar  Aamir Khan Productions             NaN   

      foreign_gross  year  
210       3300000.0  2010  
555       3300000.0  2011  
933       4000000.0  2012  
1862            NaN  2014  
2825    122000000.0  2017  


In [26]:
# Dropping rows where domestic gross is null
movie_gross.dropna(subset=['domestic_gross'], inplace=True)

In [27]:
movie_gross.isnull().sum()

title                0
studio               0
domestic_gross       0
foreign_gross     1350
year                 0
dtype: int64

In [28]:
# Creating another dataset with rows where foreign gross is not null for possible separate analysis later on
foreign_gross_not_null = movie_gross.dropna(subset=['foreign_gross'])

In [29]:
# Standardizing names in studio and title
def standardize_text(text):
    return text.strip().lower()

movie_gross['title'] = movie_gross['title'].apply(standardize_text)
movie_gross['studio'] = movie_gross['studio'].apply(standardize_text)
foreign_gross_not_null['title'] = foreign_gross_not_null['title'].apply(standardize_text)
foreign_gross_not_null['studio'] = foreign_gross_not_null['studio'].apply(standardize_text)

print(movie_gross.head())
print(foreign_gross_not_null.head())


                                         title studio  domestic_gross  \
0                                  toy story 3     bv     415000000.0   
1                   alice in wonderland (2010)     bv     334200000.0   
2  harry potter and the deathly hallows part 1     wb     296000000.0   
3                                    inception     wb     292600000.0   
4                          shrek forever after   p/dw     238700000.0   

   foreign_gross  year  
0    652000000.0  2010  
1    691300000.0  2010  
2    664300000.0  2010  
3    535700000.0  2010  
4    513900000.0  2010  
                                         title studio  domestic_gross  \
0                                  toy story 3     bv     415000000.0   
1                   alice in wonderland (2010)     bv     334200000.0   
2  harry potter and the deathly hallows part 1     wb     296000000.0   
3                                    inception     wb     292600000.0   
4                          shrek forever after

<ipython-input-29-dcc141885058>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  foreign_gross_not_null['title'] = foreign_gross_not_null['title'].apply(standardize_text)
<ipython-input-29-dcc141885058>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  foreign_gross_not_null['studio'] = foreign_gross_not_null['studio'].apply(standardize_text)


In [30]:
#Checking for duplicates in both datasets
duplicates_movie_gross = movie_gross.duplicated().sum()
print(f"Number of duplicate rows in movie_gross: {duplicates_movie_gross}")

if duplicates_movie_gross > 0:
    print("Duplicate rows in movie_gross:")
    print(movie_gross[movie_gross.duplicated()])

duplicates_foreign_gross_not_null = foreign_gross_not_null.duplicated().sum()
print(f"Number of duplicate rows in foreign_gross_not_null: {duplicates_foreign_gross_not_null}")

if duplicates_foreign_gross_not_null > 0:
    print("Duplicate rows in foreign_gross_not_null:")
    print(foreign_gross_not_null[foreign_gross_not_null.duplicated()])


Number of duplicate rows in movie_gross: 0
Number of duplicate rows in foreign_gross_not_null: 0


### Investigating and Merging the IMDB Dataset

In [31]:
print(tables)

            name
0   movie_basics
1      directors
2      known_for
3     movie_akas
4  movie_ratings
5        persons
6     principals
7        writers


In [38]:
# Checking if primary_title could be the key linking this table to movie_gross
movie_basics = pd.read_sql_query("SELECT * FROM movie_basics", conn)

print(movie_basics.head())

    movie_id                    primary_title              original_title  \
0  tt0063540                        Sunghursh                   Sunghursh   
1  tt0066787  One Day Before the Rainy Season             Ashad Ka Ek Din   
2  tt0069049       The Other Side of the Wind  The Other Side of the Wind   
3  tt0069204                  Sabse Bada Sukh             Sabse Bada Sukh   
4  tt0100275         The Wandering Soap Opera       La Telenovela Errante   

   start_year  runtime_minutes                genres  
0        2013            175.0    Action,Crime,Drama  
1        2019            114.0       Biography,Drama  
2        2018            122.0                 Drama  
3        2018              NaN          Comedy,Drama  
4        2017             80.0  Comedy,Drama,Fantasy  


In [49]:
# Standardizing primary title so that it can be used for a join
def standardize_text(text):
    return text.strip().lower()

movie_basics['primary_title'] = movie_basics['primary_title'].apply(standardize_text)

print(movie_basics.head())


    movie_id                    primary_title              original_title  \
0  tt0063540                        sunghursh                   Sunghursh   
1  tt0066787  one day before the rainy season             Ashad Ka Ek Din   
2  tt0069049       the other side of the wind  The Other Side of the Wind   
3  tt0069204                  sabse bada sukh             Sabse Bada Sukh   
4  tt0100275         the wandering soap opera       La Telenovela Errante   

   start_year  runtime_minutes                genres  
0        2013            175.0    Action,Crime,Drama  
1        2019            114.0       Biography,Drama  
2        2018            122.0                 Drama  
3        2018              NaN          Comedy,Drama  
4        2017             80.0  Comedy,Drama,Fantasy  


In [60]:
# Loading in the other tables
conn = sqlite3.connect('im.db')
movie_basics = pd.read_sql_query("SELECT * FROM movie_basics", conn)
movie_ratings = pd.read_sql_query("SELECT * FROM movie_ratings", conn)
movie_akas = pd.read_sql_query("SELECT * FROM movie_akas", conn)
writers = pd.read_sql_query("SELECT * FROM writers", conn)
directors = pd.read_sql_query("SELECT * FROM directors", conn)
persons = pd.read_sql_query("SELECT * FROM persons", conn)

In [56]:
movie_ratings.head()

,movie_id,averagerating,numvotes
0,tt10356526,8.3,31
1,tt10384606,8.9,559
2,tt1042974,6.4,20
3,tt1043726,4.2,50352
4,tt1060240,6.5,21


In [57]:
movie_akas.head()

,movie_id,ordering,title,region,language,types,attributes,is_original_title
0,tt0369610,10,Джурасик свят,BG,bg,None,None,0.0
1,tt0369610,11,Jurashikku warudo,JP,None,imdbDisplay,None,0.0
2,tt0369610,12,Jurassic World: O Mundo dos Dinossauros,BR,None,imdbDisplay,None,0.0
3,tt0369610,13,O Mundo dos Dinossauros,BR,None,None,short title,0.0
4,tt0369610,14,Jurassic World,FR,None,imdbDisplay,None,0.0


In [83]:
# Define and execute the SQL query to create the temporary table
query_create_temp_movie_basics = """
CREATE TEMP TABLE IF NOT EXISTS temp_movie_basics AS
SELECT 
    mb.movie_id,
    mb.primary_title,
    mb.start_year,
    mb.runtime_minutes,
    mb.genres,
    mr.averagerating
FROM 
    movie_basics mb
LEFT JOIN 
    movie_ratings mr ON mb.movie_id = mr.movie_id;
"""

# Execute the query
conn.execute(query_create_temp_movie_basics)

# Commit the changes
conn.commit()


In [84]:
# Define the SQL query to read the first DataFrame
query_read_first_df = """
SELECT 
    movie_id,
    primary_title,
    runtime_minutes,
    genres,
    averagerating
FROM 
    temp_movie_basics;
"""

# Execute the query and load the result into a pandas DataFrame
first_df = pd.read_sql_query(query_read_first_df, conn)

# Display the first few rows of the resulting DataFrame
print(first_df.head())


    movie_id                    primary_title  runtime_minutes  \
0  tt0063540                        Sunghursh            175.0   
1  tt0063540                        Sunghursh            175.0   
2  tt0063540                        Sunghursh            175.0   
3  tt0063540                        Sunghursh            175.0   
4  tt0066787  One Day Before the Rainy Season            114.0   

               genres  averagerating  
0  Action,Crime,Drama            7.0  
1  Action,Crime,Drama            7.0  
2  Action,Crime,Drama            7.0  
3  Action,Crime,Drama            7.0  
4     Biography,Drama            7.2  


In [94]:
conn.close()

In [114]:
import pandas as pd

# Standardize the title for joining purposes
def standardize_text(text):
    return text.strip().lower() if isinstance(text, str) else text

first_df['primary_title'] = first_df['primary_title'].apply(standardize_text)
movie_gross['title'] = movie_gross['title'].apply(standardize_text)

# Ensure there are no NaN values in the keys
first_df.dropna(subset=['primary_title'], inplace=True)
movie_gross.dropna(subset=['title'], inplace=True)

# Drop duplicates based on the merge keys
first_df.drop_duplicates(subset=['movie_id', 'primary_title'], inplace=True)
movie_gross.drop_duplicates(subset=['title'], inplace=True)

print("First DataFrame shape after dropping duplicates:", first_df.shape)
print("Movie Gross DataFrame shape after dropping duplicates:", movie_gross.shape)


First DataFrame shape after dropping duplicates: (146144, 5)
Movie Gross DataFrame shape after dropping duplicates: (3358, 5)


In [115]:
# Perform the full outer merge
full_df = pd.merge(first_df, movie_gross, left_on='primary_title', right_on='title', how='outer')

# Display the shape and first few rows to verify
print("Full DataFrame shape after merge:", full_df.shape)
print(full_df.head())

Full DataFrame shape after merge: (146819, 10)
    movie_id                    primary_title  runtime_minutes  \
0  tt0063540                        sunghursh            175.0   
1  tt0066787  one day before the rainy season            114.0   
2  tt0069049       the other side of the wind            122.0   
3  tt0069204                  sabse bada sukh              NaN   
4  tt0100275         the wandering soap opera             80.0   

                 genres  averagerating title studio  domestic_gross  \
0    Action,Crime,Drama            7.0   NaN    NaN             NaN   
1       Biography,Drama            7.2   NaN    NaN             NaN   
2                 Drama            6.9   NaN    NaN             NaN   
3          Comedy,Drama            6.1   NaN    NaN             NaN   
4  Comedy,Drama,Fantasy            6.5   NaN    NaN             NaN   

   foreign_gross  year  
0            NaN   NaN  
1            NaN   NaN  
2            NaN   NaN  
3            NaN   NaN  
4   

In [116]:
full_df.isnull().sum()

movie_id              675
primary_title         675
runtime_minutes     32414
genres               6083
averagerating       72963
title              142684
studio             142684
domestic_gross     142684
foreign_gross      144325
year               142684
dtype: int64

In [117]:
# Rows where primary_title is null and title is not null
null_primary_not_null_title = full_df[full_df['primary_title'].isnull() & full_df['title'].notnull()]

# Display the result
print("Rows where primary_title is null and title is not null:")
print(null_primary_not_null_title.head())
print("Number of such rows:", null_primary_not_null_title.shape[0])


Rows where primary_title is null and title is not null:
       movie_id primary_title  runtime_minutes genres  averagerating  \
146144      NaN           NaN              NaN    NaN            NaN   
146145      NaN           NaN              NaN    NaN            NaN   
146146      NaN           NaN              NaN    NaN            NaN   
146147      NaN           NaN              NaN    NaN            NaN   
146148      NaN           NaN              NaN    NaN            NaN   

                                              title studio  domestic_gross  \
146144                   alice in wonderland (2010)     bv     334200000.0   
146145  harry potter and the deathly hallows part 1     wb     296000000.0   
146146                   clash of the titans (2010)     wb     163200000.0   
146147                                  tron legacy     bv     172100000.0   
146148                                 knight & day    fox      76400000.0   

        foreign_gross    year  
146144    

In [118]:
# Fill missing primary_title with title
full_df['primary_title'] = full_df['primary_title'].fillna(full_df['title'])

# Fill missing title with primary_title
full_df['title'] = full_df['title'].fillna(full_df['primary_title'])

# Display the first few rows to verify
print(full_df.head())

# Check for any remaining missing values in key columns
missing_values = full_df[['primary_title', 'title']].isnull().sum()
print("Missing values in key columns after filling:\n", missing_values)

    movie_id                    primary_title  runtime_minutes  \
0  tt0063540                        sunghursh            175.0   
1  tt0066787  one day before the rainy season            114.0   
2  tt0069049       the other side of the wind            122.0   
3  tt0069204                  sabse bada sukh              NaN   
4  tt0100275         the wandering soap opera             80.0   

                 genres  averagerating                            title  \
0    Action,Crime,Drama            7.0                        sunghursh   
1       Biography,Drama            7.2  one day before the rainy season   
2                 Drama            6.9       the other side of the wind   
3          Comedy,Drama            6.1                  sabse bada sukh   
4  Comedy,Drama,Fantasy            6.5         the wandering soap opera   

  studio  domestic_gross  foreign_gross  year  
0    NaN             NaN            NaN   NaN  
1    NaN             NaN            NaN   NaN  
2    NaN

In [119]:
full_df.isnull().sum()

movie_id              675
primary_title           0
runtime_minutes     32414
genres               6083
averagerating       72963
title                   0
studio             142684
domestic_gross     142684
foreign_gross      144325
year               142684
dtype: int64

In [121]:
writers

,movie_id,person_id,writer_name
0,tt0285252,nm0899854,Tony Vitale
1,tt0438973,nm0175726,Steve Conrad
2,tt0438973,nm1802864,Sean Sorensen
3,tt0462036,nm1940585,Bill Haley
4,tt0835418,nm0310087,Peter Gaulke
...,...,...,...
255868,tt8999892,nm10122246,Bradley T. Castle
255869,tt8999974,nm10122357,Daysi Burbano
255870,tt9001390,nm6711477,Bernard Lessa
255871,tt9004986,nm4993825,Fredrik Horn Akselsen


In [125]:
# Adding writers to the df
writers['writer_name'] = writers['writer_name'].fillna('unknown')

# Aggregate writer names for each movie_id
writers_grouped = writers.groupby('movie_id')['writer_name'].agg(lambda x: ', '.join(x)).reset_index()

# Rename the aggregated column to 'writers'
writers_grouped.rename(columns={'writer_name': 'writers'}, inplace=True)

# Display the first few rows to verify
print(writers_grouped.head())

# Merge the writers_grouped with full_df on 'movie_id'
full_df = pd.merge(full_df, writers_grouped, on='movie_id', how='left')

# Display the first few rows to verify
full_df.head()

    movie_id                                            writers
0  tt0063540  Abrar Alvi, Mahasweta Devi, Gulzar, Anjana Rawail
1  tt0069049                            Orson Welles, Oja Kodar
2  tt0069204                                             Gulzar
3  tt0100275           Pía Rey, Pía Rey, Raoul Ruiz, Raoul Ruiz
4  tt0111414                                       Frank Howson


,movie_id,primary_title,runtime_minutes,genres,averagerating,title,studio,domestic_gross,foreign_gross,year,writers_x,writers_y
0,tt0063540,sunghursh,175.0,"Action,Crime,Drama",7.0,sunghursh,NaN,NaN,NaN,NaN,"Abrar Alvi, Mahasweta Devi, Gulzar, Anjana Rawail","Abrar Alvi, Mahasweta Devi, Gulzar, Anjana Rawail"
1,tt0066787,one day before the rainy season,114.0,"Biography,Drama",7.2,one day before the rainy season,NaN,NaN,NaN,NaN,NaN,NaN
2,tt0069049,the other side of the wind,122.0,Drama,6.9,the other side of the wind,NaN,NaN,NaN,NaN,"Orson Welles, Oja Kodar","Orson Welles, Oja Kodar"
3,tt0069204,sabse bada sukh,NaN,"Comedy,Drama",6.1,sabse bada sukh,NaN,NaN,NaN,NaN,Gulzar,Gulzar
4,tt0100275,the wandering soap opera,80.0,"Comedy,Drama,Fantasy",6.5,the wandering soap opera,NaN,NaN,NaN,NaN,"Pía Rey, Pía Rey, Raoul Ruiz, Raoul Ruiz","Pía Rey, Pía Rey, Raoul Ruiz, Raoul Ruiz"


In [126]:
full_df.columns

Index(['movie_id', 'primary_title', 'runtime_minutes', 'genres',
       'averagerating', 'title', 'studio', 'domestic_gross', 'foreign_gross',
       'year', 'writers_x', 'writers_y'],
      dtype='object')

In [127]:
# Checking if writers_x is identical to writers_y
if 'writers_x' in full_df.columns and 'writers_y' in full_df.columns:
    are_identical = full_df['writers_x'].equals(full_df['writers_y'])
    print("Are writers_x and writers_y identical?", are_identical)
else:
    print("Either writers_x or writers_y column does not exist.")

Are writers_x and writers_y identical? True


In [128]:
full_df.drop(columns=['writers_y'], inplace=True)

In [130]:
full_df.head(20)

,movie_id,primary_title,runtime_minutes,genres,averagerating,title,studio,domestic_gross,foreign_gross,year,writers_x
0,tt0063540,sunghursh,175.0,"Action,Crime,Drama",7.0,sunghursh,NaN,NaN,NaN,NaN,"Abrar Alvi, Mahasweta Devi, Gulzar, Anjana Rawail"
1,tt0066787,one day before the rainy season,114.0,"Biography,Drama",7.2,one day before the rainy season,NaN,NaN,NaN,NaN,NaN
2,tt0069049,the other side of the wind,122.0,Drama,6.9,the other side of the wind,NaN,NaN,NaN,NaN,"Orson Welles, Oja Kodar"
3,tt0069204,sabse bada sukh,NaN,"Comedy,Drama",6.1,sabse bada sukh,NaN,NaN,NaN,NaN,Gulzar
4,tt0100275,the wandering soap opera,80.0,"Comedy,Drama,Fantasy",6.5,the wandering soap opera,NaN,NaN,NaN,NaN,"Pía Rey, Pía Rey, Raoul Ruiz, Raoul Ruiz"
5,tt0111414,a thin life,75.0,Comedy,NaN,a thin life,NaN,NaN,NaN,NaN,Frank Howson
6,tt0112502,bigfoot,NaN,"Horror,Thriller",4.1,bigfoot,NaN,NaN,NaN,NaN,NaN
7,tt4503112,bigfoot,NaN,"Action,Horror",4.6,bigfoot,NaN,NaN,NaN,NaN,"Danette Bradford, Rocky Burnswork"
8,tt9181914,bigfoot,86.0,"Animation,Family",2.5,bigfoot,NaN,NaN,NaN,NaN,Holly May Opee
9,tt0137204,joe finds grace,83.0,"Adventure,Animation,Comedy",8.1,joe finds grace,NaN,NaN,NaN,NaN,Anthony Harrison


In [131]:
full_df.isnull().sum()

movie_id              675
primary_title           0
runtime_minutes     32414
genres               6083
averagerating       72963
title                   0
studio             142684
domestic_gross     142684
foreign_gross      144325
year               142684
writers_x           36558
dtype: int64

In [132]:
# Replace None values with 'unknown' in director_name
directors['director_name'] = directors['director_name'].fillna('unknown')

# Aggregate director names for each movie_id
directors_grouped = directors.groupby('movie_id')['director_name'].agg(lambda x: ', '.join(x)).reset_index()

# Rename the aggregated column to 'directors'
directors_grouped.rename(columns={'director_name': 'directors'}, inplace=True)

# Display the first few rows to verify
print(directors_grouped.head())

# Merge the directors_grouped with full_df on 'movie_id'
full_df = pd.merge(full_df, directors_grouped, on='movie_id', how='left')

# Display the first few rows to verify
print(full_df.head())

full_df.head()

    movie_id                                          directors
0  tt0063540  Harnam Singh Rawail, Harnam Singh Rawail, Harn...
1  tt0066787                                          Mani Kaul
2  tt0069049                         Orson Welles, Orson Welles
3  tt0069204                               Hrishikesh Mukherjee
4  tt0100275  Valeria Sarmiento, Raoul Ruiz, Valeria Sarmien...
    movie_id                    primary_title  runtime_minutes  \
0  tt0063540                        sunghursh            175.0   
1  tt0066787  one day before the rainy season            114.0   
2  tt0069049       the other side of the wind            122.0   
3  tt0069204                  sabse bada sukh              NaN   
4  tt0100275         the wandering soap opera             80.0   

                 genres  averagerating                            title  \
0    Action,Crime,Drama            7.0                        sunghursh   
1       Biography,Drama            7.2  one day before the rainy seas

,movie_id,primary_title,runtime_minutes,genres,averagerating,title,studio,domestic_gross,foreign_gross,year,writers_x,directors
0,tt0063540,sunghursh,175.0,"Action,Crime,Drama",7.0,sunghursh,NaN,NaN,NaN,NaN,"Abrar Alvi, Mahasweta Devi, Gulzar, Anjana Rawail","Harnam Singh Rawail, Harnam Singh Rawail, Harn..."
1,tt0066787,one day before the rainy season,114.0,"Biography,Drama",7.2,one day before the rainy season,NaN,NaN,NaN,NaN,NaN,Mani Kaul
2,tt0069049,the other side of the wind,122.0,Drama,6.9,the other side of the wind,NaN,NaN,NaN,NaN,"Orson Welles, Oja Kodar","Orson Welles, Orson Welles"
3,tt0069204,sabse bada sukh,NaN,"Comedy,Drama",6.1,sabse bada sukh,NaN,NaN,NaN,NaN,Gulzar,Hrishikesh Mukherjee
4,tt0100275,the wandering soap opera,80.0,"Comedy,Drama,Fantasy",6.5,the wandering soap opera,NaN,NaN,NaN,NaN,"Pía Rey, Pía Rey, Raoul Ruiz, Raoul Ruiz","Valeria Sarmiento, Raoul Ruiz, Valeria Sarmien..."


In [134]:
full_df.isnull().sum()

movie_id              675
primary_title           0
runtime_minutes     32414
genres               6083
averagerating       72963
title                   0
studio             142684
domestic_gross     142684
foreign_gross      144325
year               142684
writers_x           36558
directors            6402
dtype: int64

In [138]:
# Standardizing names for directors and writers

# Define a function to standardize text
def standardize_text(text):
    return text.strip().lower() if isinstance(text, str) else text

# Define a function to standardize names separated by commas
def standardize_names(names):
    if isinstance(names, str):
        return ', '.join([standardize_text(name) for name in names.split(',')])
    return names

# Apply the function to standardize names in 'directors'
full_df['directors'] = full_df['directors'].apply(standardize_names)

# Apply the function to standardize names in 'writers'
full_df['writers_x'] = full_df['writers_x'].apply(standardize_names)

# Display the first few rows to verify
full_df[['directors', 'writers_x']].head()

,directors,writers_x
0,"harnam singh rawail, harnam singh rawail, harn...","abrar alvi, mahasweta devi, gulzar, anjana rawail"
1,mani kaul,NaN
2,"orson welles, orson welles","orson welles, oja kodar"
3,hrishikesh mukherjee,gulzar
4,"valeria sarmiento, raoul ruiz, valeria sarmien...","pía rey, pía rey, raoul ruiz, raoul ruiz"


In [141]:
#Removing copies of names
def remove_duplicates(names):
    if isinstance(names, str):
        unique_names = set()
        standardized_names = []
        for name in names.split(','):
            name = name.strip()
            if name not in unique_names:
                standardized_names.append(name)
                unique_names.add(name)
        return ', '.join(standardized_names)
    return names

# Apply the function to remove duplicates in 'directors'
full_df['directors'] = full_df['directors'].apply(remove_duplicates)

# Apply the function to remove duplicates in 'writers'
full_df['writers_x'] = full_df['writers_x'].apply(remove_duplicates)

# Display the first few rows to verify
print(full_df[['directors', 'writers_x']].head())

                       directors  \
0            harnam singh rawail   
1                      mani kaul   
2                   orson welles   
3           hrishikesh mukherjee   
4  valeria sarmiento, raoul ruiz   

                                           writers_x  
0  abrar alvi, mahasweta devi, gulzar, anjana rawail  
1                                                NaN  
2                            orson welles, oja kodar  
3                                             gulzar  
4                                pía rey, raoul ruiz  


In [142]:
full_df.head(10)

,movie_id,primary_title,runtime_minutes,genres,averagerating,title,studio,domestic_gross,foreign_gross,year,writers_x,directors
0,tt0063540,sunghursh,175.0,"Action,Crime,Drama",7.0,sunghursh,NaN,NaN,NaN,NaN,"abrar alvi, mahasweta devi, gulzar, anjana rawail",harnam singh rawail
1,tt0066787,one day before the rainy season,114.0,"Biography,Drama",7.2,one day before the rainy season,NaN,NaN,NaN,NaN,NaN,mani kaul
2,tt0069049,the other side of the wind,122.0,Drama,6.9,the other side of the wind,NaN,NaN,NaN,NaN,"orson welles, oja kodar",orson welles
3,tt0069204,sabse bada sukh,NaN,"Comedy,Drama",6.1,sabse bada sukh,NaN,NaN,NaN,NaN,gulzar,hrishikesh mukherjee
4,tt0100275,the wandering soap opera,80.0,"Comedy,Drama,Fantasy",6.5,the wandering soap opera,NaN,NaN,NaN,NaN,"pía rey, raoul ruiz","valeria sarmiento, raoul ruiz"
5,tt0111414,a thin life,75.0,Comedy,NaN,a thin life,NaN,NaN,NaN,NaN,frank howson,frank howson
6,tt0112502,bigfoot,NaN,"Horror,Thriller",4.1,bigfoot,NaN,NaN,NaN,NaN,NaN,mc jones
7,tt4503112,bigfoot,NaN,"Action,Horror",4.6,bigfoot,NaN,NaN,NaN,NaN,"danette bradford, rocky burnswork",david heavener
8,tt9181914,bigfoot,86.0,"Animation,Family",2.5,bigfoot,NaN,NaN,NaN,NaN,holly may opee,evan tramel
9,tt0137204,joe finds grace,83.0,"Adventure,Animation,Comedy",8.1,joe finds grace,NaN,NaN,NaN,NaN,anthony harrison,anthony harrison
